In [5]:
import os
import shutil
import subprocess
import logging
from time import time
import multiprocessing
from pprint import pprint
import collections

import json
import pydicom
import numpy as np
from scipy.sparse import csc_matrix
import math
import matplotlib.pyplot as plt
import numpy.matlib
from scipy import interpolate
from datetime import datetime

import os.path as osp
import tempfile
from unittest import TestCase

import matplotlib.pyplot as plt
import numpy as np

from pylinac.core.geometry import Point
from pylinac import Starshot
from tests_basic.utils import save_file, LoadingTestBase, LocationMixin

plt.close('all')
TEST_DIR = osp.join('/', 'test_files', 'Starshot')

SOPClassUIDs = {
    'CR': [
        '1.2.840.10008.5.1.4.1.1.1', #CR Storage 
        '1.2.840.10008.5.1.4.1.1.1.1', #CR Presentation 
        '1.2.840.10008.5.1.4.1.1.1.1.1' #CR Processing
    ],
    'XC': [
        '1.2.840.10008.5.1.4.1.1.77.1.4' #CR Storage 
    ],
 
}



class StarshotRead:
    def __init__(self, InFolder):
        day = datetime.today().strftime('%Y/%m/%d')
        base_path = "/var/www/public/uploads/dcmtk/"+day
        tmp = InFolder.rpartition('/') # seriesUidHash
        tmp2 = tmp[0].rpartition('/')  # studyUidHash

        self.InFolder = InFolder
        self.BasePath = base_path + '/' + tmp2[2] + '/' + tmp[2]
        kwargs = {'dpi': 30, 'sid': 1000}
        
        self.seriesInstanceUid = ''
        self.fileString = ''
 
        
    def load_files(self):
        ret = []
        for dirName, subdirList, fileList in os.walk(self.InFolder):
            for filename in fileList:
            
                file = os.path.join(dirName, filename)
                ret.append(file)
                tmp_file = pydicom.dcmread(file, force=True, specific_tags=['SOPClassUID'])
                if "PatientsName" in tmp_file:
               
                    if tmp_file.SOPClassUID in SOPClassUIDs['CR']:
                        ret.append(file)

                       #self.seriesInstanceUid = tmp_file.SeriesInstanceUID
                    elif tmp_file.SOPClassUID in SOPClassUIDs['XC']:
                        ret.append(file)
                       # self.seriesInstanceUid = tmp_file.SeriesInstanceUID
                    else:
                        break
        
        self.fileString = ' '.join(ret)
        return ret
    
    def getParameters(self):
        files = self.load_files()
        for file in files:
            print("____________________________________________________________________")
            print("LINKS:")
            print('http://192.168.10.195:1515/files/pylinac/tests_basic/'+file+'.png')
            print("Abrir pdf e salvar pc para visualizar, pelo navegador pdf não abre!!")
            print('http://192.168.10.195:1515/files/pylinac/tests_basic/'+file+'.pdf')
            print("RESULTADOS:")
            mystar = Starshot(file, dpi=300 ,sid=1000)
            mystar.analyze(radius=0.85, min_peak_height=0.25, tolerance=2, fwhm = True, recursive = True, invert=False)
            print(mystar.results())
            mystar.plot_analyzed_image()
            mystar.save_analyzed_image(file+'.png')
            mystar.publish_pdf(file+'.pdf')
           


     

mpr = StarshotRead('test_files/Starshot/')
mpr.getParameters()


____________________________________________________________________
LINKS:
http://192.168.10.195:1515/files/pylinac/tests_basic/test_files/Starshot/gantry.dcm.png
Abrir pdf e salvar pc para visualizar, pelo navegador pdf não abre!!
http://192.168.10.195:1515/files/pylinac/tests_basic/test_files/Starshot/gantry.dcm.pdf
RESULTADOS:


RuntimeError: The algorithm was unable to determine a reasonable wobble. Try setting recursive to False and manually adjusting algorithm parameters